<a href="https://colab.research.google.com/github/swarthyPig/Raspberry-Pi_DHT22_Weather_Station_with_correlation-and-ML/blob/master/machine%20learning/Multi_Variable_TF_MLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.parsers import read_csv

model = tf.global_variables_initializer();

 
data = read_csv('iot_data.csv', sep=',')

# date는 data type이 float이 아니므로 오류발생해서 삭제했습니다.
# csv 파일안의 행 순서는 index_number, temp, humi, cr 입니다.

xy = np.array(data, dtype=np.float32)

x_data = xy[0:-50, 1:-1] # temp, humi

y_data = xy[0:-50, [-1]] # cr

x_data_test = xy[-50:, 1:-1] # test data temp, humi

y_data_test = xy[-50:, [-1]] # test data cr


# 행렬의 Shape을 None으로 지정하는 것은 N개의 데이터를 받겠다는 의미이다.(N x 2 행렬)

X = tf.placeholder(tf.float32, shape=[None, 2])

Y = tf.placeholder(tf.float32, shape=[None, 1])


# 각 변수를 array가 아닌 matrix로 나타냅니다.

W = tf.Variable(tf.random_normal([2, 1]), name="weight")

b = tf.Variable(tf.random_normal([1]), name="bias")


# matrix로 나타낸 변수를 행렬곱을 사용하여 가설을 세웁니다.

hypothesis = tf.matmul(X, W) + b


# 비용 함수를 설정합니다.

cost = tf.reduce_mean(tf.square(hypothesis - Y))


# 경사하강 라이브러리
# 최적화 함수를 설정합니다.

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)

train = optimizer.minimize(cost)



# 세션을 생성합니다.

sess = tf.Session()



# 글로벌 변수를 초기화합니다.

sess.run(tf.global_variables_initializer())



# 학습을 수행합니다.

for step in range(300001):

    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})

    if step % 10000 == 0:

        print("#", step, " 손실 비용: ", cost_)
        print("- Weigth : ", sess.run(W[0]),sess.run(W[1]))
        print("- Bias : ", sess.run(b))
        print("- 쾌적도: ", hypo_[0])
        

saver = tf.train.Saver()

save_path = saver.save(sess, "saved.cpkt")

print('학습된 모델을 저장했습니다.')




# 0  손실 비용:  132.55804
- Weigth :  [0.6650802] [-0.3934198]
- Bias :  [0.929505]
- 쾌적도:  [-10.759264]
# 10000  손실 비용:  2.1324463
- Weigth :  [-0.06131026] [0.08272044]
- Bias :  [1.0872298]
- 쾌적도:  [3.9767084]
# 20000  손실 비용:  2.1005902
- Weigth :  [-0.06730674] [0.0819829]
- Bias :  [1.2655901]
- 쾌적도:  [3.9768503]
# 30000  손실 비용:  2.0692596
- Weigth :  [-0.07325644] [0.08125222]
- Bias :  [1.442506]
- 쾌적도:  [3.9769974]
# 40000  손실 비용:  2.0384467
- Weigth :  [-0.07915545] [0.08052628]
- Bias :  [1.617985]
- 쾌적도:  [3.9771352]
# 50000  손실 비용:  2.0081837
- Weigth :  [-0.08500152] [0.07980868]
- Bias :  [1.7918031]
- 쾌적도:  [3.9772806]
# 60000  손실 비용:  1.9784223
- Weigth :  [-0.09079919] [0.07909661]
- Bias :  [1.9642015]
- 쾌적도:  [3.9774237]
# 70000  손실 비용:  1.9491035
- Weigth :  [-0.09655987] [0.07838935]
- Bias :  [2.1354876]
- 쾌적도:  [3.9775667]
# 80000  손실 비용:  1.9203739
- Weigth :  [-0.10224996] [0.07768884]
- Bias :  [2.3047647]
- 쾌적도:  [3.9776978]
# 90000  손실 비용:  1.8920889
- Weigth :

In [0]:
# 온,습도 변수의 값을 입력 받습니다.

temperature = float(input('온도: '))

humidity = float(input('습도: '))


with tf.Session() as sess:

    sess.run(model)


    # 저장된 학습 모델을 파일로부터 불러옵니다.
    
    save_path = "saved.cpkt"
    
    saver.restore(sess, save_path)



    # 사용자의 입력 값을 이용해 배열을 만듭니다.
    
    data = ((temperature, humidity), )
    
    arr = np.array(data, dtype=np.float32)



    # 예측을 수행한 뒤에 그 결과를 출력합니다.
    
    x_data = arr[0:2]
    
    dict = sess.run(hypothesis, feed_dict={X: x_data})

    print(dict[0])
    
    # Show the values of weights and bias
    print('Weights:')
    print(sess.run(W))
    print('Bias:')
    print(sess.run(b))

온도: 28
습도: 43
INFO:tensorflow:Restoring parameters from saved.cpkt
[2.378653]
Weights:
[[-0.2161315 ]
 [ 0.06369445]]
Bias:
[5.6914735]
